In [1]:
import os

if not os.path.exists("VoxCeleb_gender"):
    !pip install gdown pydub librosa
    !gdown https://drive.google.com/uc?id=1V3lNnSDv71AJYIj71V4_xD0ir0DrXxEV
    !tar -xf VoxCeleb_gender.zip #!tar -xf VoxCeleb_gender.zip
    !del VoxCeleb_gender.zip     #!del

else:
    print("Dataset already downloaded.")

Dataset already downloaded.


In [2]:
import librosa
import scipy.signal as ss
import numpy as np
import pandas as pd
import os

def extract_mfcc(file_path, n_mfcc=12, win_length=320, hop_length=160):
    wave, sr = librosa.load(file_path, mono=True, sr=None)
    if sr != 16000:
        wave = librosa.resample(wave, sr, 16000)
    mfcc = librosa.feature.mfcc(y=wave, sr=16000, n_mfcc=n_mfcc, window=ss.windows.hamming, win_length=win_length, hop_length=hop_length)
    return mfcc

# Build data in memory

In [3]:
mfcc_coeff_amount = 12
window_length = int(16000*0.02)

In [4]:
arr = []
if not os.path.exists(f"mfcc_{mfcc_coeff_amount}_{window_length}.pkl"):

    aux = []

    for (dirpath, dirnames, filenames) in os.walk("VoxCeleb_gender"):
        for filename in filenames:
            if filename.endswith(".wav"):
                filepath = dirpath + "/" + filename
                aux.append({"file": filepath, "gender": dirpath.split("/")[-1]})

    df = pd.DataFrame(aux)

    df["mfcc"] = pd.Series(name="mfcc", dtype=object)

    for i in range(len(df)):
        mfcc_coeffs = extract_mfcc(df.loc[i, "file"], n_mfcc=mfcc_coeff_amount, win_length=window_length)
        for i, coeffs in enumerate(mfcc_coeffs):
            if np.linalg.norm(coeffs) < 1:
                mfcc_coeffs.pop(i)
        arr.append(mfcc_coeffs)
    df["mfcc"] = arr
    df.to_pickle(f"mfcc_{mfcc_coeff_amount}_{window_length}")
else:
    print("File Found")
    df = pd.read_pickle(f"mfcc_{mfcc_coeff_amount}_{window_length}.pkl")


num_males = len(df.loc[df.gender == "males"] )           
num_females = len(df.loc[df.gender == "females"] )              

File Found


In [5]:
print("Number of males: ", num_males)
print("Number of females: ", num_females)

Number of males:  3682
Number of females:  2311


In [6]:
display(df)

,file,gender,mfcc
0,VoxCeleb_gender/males/255.wav,males,"[[-266.89233, -288.64322, -288.0051, -277.8506..."
1,VoxCeleb_gender/males/1908.wav,males,"[[-362.22406, -338.4697, -335.41837, -333.9136..."
2,VoxCeleb_gender/males/2965.wav,males,"[[-407.10593, -375.89368, -386.202, -388.78607..."
3,VoxCeleb_gender/males/2733.wav,males,"[[-299.1009, -257.6437, -261.46454, -229.911, ..."
4,VoxCeleb_gender/males/3091.wav,males,"[[-259.80472, -288.65842, -285.53375, -291.276..."
...,...,...,...
5988,VoxCeleb_gender/females/489.wav,females,"[[-439.62015, -451.93927, -449.33908, -452.043..."
5989,VoxCeleb_gender/females/941.wav,females,"[[-327.78085, -321.0226, -342.2135, -330.92526..."
5990,VoxCeleb_gender/females/63.wav,females,"[[-408.27185, -391.82303, -360.8797, -308.7707..."
5991,VoxCeleb_gender/females/1820.wav,females,"[[-365.03598, -326.88446, -337.80292, -328.621..."


# EDA

In [13]:
# Get the mean mfcc coefficients for each class
mean_mfcc_males = 0
mean_mfcc_females = 0


for person in df.iterrows():
    for i in range(len(person[1]["mfcc"])//mfcc_coeff_amount):
        if person[1]["gender"] == "males":
            mean_mfcc_males = np.mean(np.array(person[1]["mfcc"]), axis=1)/num_males
        else:
            mean_mfcc_females = np.mean(person[1]["mfcc"], axis=1)/num_females

display(mean_mfcc_males)
display(mean_mfcc_females)

array([-0.11972281,  0.02990563, -0.01197125,  0.00156714, -0.0089404 ,
       -0.00567426, -0.00416346, -0.00176591, -0.00151322, -0.00049492,
       -0.00278864,  0.00304949], dtype=float32)

array([-0.1156978 ,  0.03775504, -0.00754318,  0.00508091, -0.01344521,
       -0.01013145, -0.00357516, -0.00078147,  0.00039397,  0.00086333,
       -0.0044957 , -0.00069347], dtype=float32)

In [ ]:
for i in range(100):
    